In [46]:
import pandas as pd
import numpy as np
import sqlite3

# Reconnect to the database
conn = sqlite3.connect('./ipl_database.db')
# Query the database
df_ball = pd.read_sql_query('SELECT * FROM ipl_ball_by_ball;', conn)

# Display the first few rows of the table
print(df_ball)

# Close the connection
conn.close()


             ID  innings  overs  ballnumber       batter          bowler   
0       1312200        1      0           1  YBK Jaiswal  Mohammed Shami  \
1       1312200        1      0           2  YBK Jaiswal  Mohammed Shami   
2       1312200        1      0           3   JC Buttler  Mohammed Shami   
3       1312200        1      0           4  YBK Jaiswal  Mohammed Shami   
4       1312200        1      0           5  YBK Jaiswal  Mohammed Shami   
...         ...      ...    ...         ...          ...             ...   
225949   335982        2     14           5      P Kumar        I Sharma   
225950   335982        2     14           6     SB Joshi        I Sharma   
225951   335982        2     14           7      P Kumar        I Sharma   
225952   335982        2     15           1     SB Joshi       LR Shukla   
225953   335982        2     15           2     SB Joshi       LR Shukla   

        non-striker extra_type  batsman_run  extras_run  total_run   
0        JC Buttl

In [47]:
unique_batters = df_ball['batter'].unique().tolist()

print(unique_batters)

['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D Padikkal', 'SO Hetmyer', 'R Ashwin', 'R Parag', 'TA Boult', 'OC McCoy', 'WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pandya', 'DA Miller', 'V Kohli', 'F du Plessis', 'RM Patidar', 'GJ Maxwell', 'MK Lomror', 'KD Karthik', 'Shahbaz Ahmed', 'PWH de Silva', 'HV Patel', 'JR Hazlewood', 'Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda', 'MP Stoinis', 'E Lewis', 'KH Pandya', 'PVD Chameera', 'PK Garg', 'Abhishek Sharma', 'RA Tripathi', 'AK Markram', 'N Pooran', 'Washington Sundar', 'R Shepherd', 'J Suchith', 'B Kumar', 'Umran Malik', 'JM Bairstow', 'S Dhawan', 'M Shahrukh Khan', 'MA Agarwal', 'LS Livingstone', 'JM Sharma', 'PN Mankad', 'PP Shaw', 'DA Warner', 'MR Marsh', 'RR Pant', 'SN Khan', 'R Powell', 'AR Patel', 'SN Thakur', 'Kuldeep Yadav', 'Ishan Kishan', 'RG Sharma', 'D Brevis', 'Tilak Varma', 'TH David', 'Ramandeep Singh', 'DR Sams', 'RD Gaikwad', 'DP Conway', 'MM Ali', 'N Jagadeesan', 'AT Rayudu', 'MS Dhoni', 'Simarjeet Singh', 'MJ Santner', 'R 

In [48]:
import sqlite3
import pandas as pd

def get_batting_stats(player_name):
    # Establish a connection to the SQLite database
    conn = sqlite3.connect('./ipl_database.db')

    # Define the SQL query with the corrected column name
    query = f'''
    SELECT
        innings,
        overs,
        SUM(CASE WHEN batter = ? THEN batsman_run ELSE 0 END) AS runs_scored,
        SUM((CASE WHEN isWicketDelivery = 1 AND player_out == ? THEN 1 ELSE 0 END)) AS Dismissals,
        COUNT(CASE WHEN (extra_type IS NULL OR extra_type = 'byes' OR extra_type = 'legbyes' OR extra_type = 'noballs') AND (batter = ?) THEN 1 ELSE NULL END) AS balls_faced
    FROM
        ipl_ball_by_ball
    WHERE
        batter = ? or "non-striker" = ?
    GROUP BY
        innings,
        overs
    ORDER BY
        innings,
        overs;
    '''

    # Execute the query with the player name as a parameter
    df = pd.read_sql_query(query, conn, params=(player_name, player_name, player_name, player_name, player_name))

    # Close the connection
    conn.close()

    return df

# Example usage
player_name = 'AB de Villiers'  # Replace with the actual player name
batter_stats_df = get_batting_stats(player_name)
batter_stats_df

,innings,overs,runs_scored,Dismissals,balls_faced
0,1,0,19,0,10
1,1,1,22,1,23
2,1,2,28,1,38
3,1,3,83,0,76
4,1,4,114,2,72
5,1,5,144,2,92
6,1,6,118,2,95
7,1,7,110,4,97
8,1,8,127,2,105
9,1,9,162,2,120


In [49]:
# Assuming you have a DataFrame named batter_stats_df
# Filter the DataFrame based on innings, overs range, and runs scored
innings_x = 1  # Specify the innings
overs_y = 0  # Specify the starting over
overs_z = 5  # Specify the ending over

filtered_batter_df = batter_stats_df[
    (batter_stats_df['innings'].isin([1, 2])) &  # Filter by innings
    (batter_stats_df['overs'] >= overs_y) &      # Filter by starting over
    (batter_stats_df['overs'] <= overs_z)
]

# Print the filtered DataFrame
filtered_batter_df

,innings,overs,runs_scored,Dismissals,balls_faced
0,1,0,19,0,10
1,1,1,22,1,23
2,1,2,28,1,38
3,1,3,83,0,76
4,1,4,114,2,72
5,1,5,144,2,92
20,2,0,3,1,7
21,2,1,18,0,16
22,2,2,33,4,37
23,2,3,53,2,46


In [50]:
runs_score = filtered_batter_df["runs_scored"].sum()
balls_faced = filtered_batter_df["balls_faced"].sum()
dismissals = filtered_batter_df["Dismissals"].sum()
average = runs_score/dismissals
strike_rate = (runs_score/balls_faced)*100
print(runs_score, balls_faced, dismissals, average, strike_rate)

722 606 22 32.81818181818182 119.14191419141915


In [51]:
def batting_stats_overs(player_name, batter_stats_df , overs_y, overs_z):

    filtered_batter_df = batter_stats_df[
        (batter_stats_df['innings'] == 1) |
        (batter_stats_df['innings'] == 2) &  # Filter by innings
        (batter_stats_df['overs'] >= overs_y) &      # Filter by starting over
        (batter_stats_df['overs'] <= overs_z)
    ]

    runs_score = filtered_batter_df["runs_scored"].sum()
    balls_faced = filtered_batter_df["balls_faced"].sum()
    dismissals = filtered_batter_df["Dismissals"].sum()
    average = runs_score/dismissals
    strike_rate = (runs_score/balls_faced)*100

    return [player_name, runs_score, balls_faced, dismissals, average, strike_rate]

In [52]:
batting_power_play = []
for i in unique_batters:
    df = get_batting_stats(i)
    batting_power_play.append(batting_stats_overs(i, df, 0, 5))

C:\Users\anubr\AppData\Local\Temp\ipykernel_1440\961564431.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  average = runs_score/dismissals
C:\Users\anubr\AppData\Local\Temp\ipykernel_1440\961564431.py:13: RuntimeWarning: invalid value encountered in scalar divide
  average = runs_score/dismissals
C:\Users\anubr\AppData\Local\Temp\ipykernel_1440\961564431.py:14: RuntimeWarning: invalid value encountered in scalar divide
  strike_rate = (runs_score/balls_faced)*100


In [53]:
batting_power_play

[['YBK Jaiswal', 452, 337, 19, 23.789473684210527, 134.1246290801187],
 ['JC Buttler', 2072, 1415, 51, 40.627450980392155, 146.43109540636044],
 ['SV Samson', 2250, 1689, 81, 27.77777777777778, 133.21492007104797],
 ['D Padikkal', 1089, 852, 38, 28.657894736842106, 127.8169014084507],
 ['SO Hetmyer', 547, 384, 19, 28.789473684210527, 142.44791666666669],
 ['R Ashwin', 375, 333, 30, 12.5, 112.61261261261262],
 ['R Parag', 328, 265, 21, 15.619047619047619, 123.77358490566037],
 ['TA Boult', 47, 34, 4, 11.75, 138.23529411764704],
 ['OC McCoy', 8, 5, 1, 8.0, 160.0],
 ['WP Saha', 1933, 1460, 72, 26.84722222222222, 132.39726027397262],
 ['Shubman Gill', 1356, 1085, 43, 31.53488372093023, 124.97695852534562],
 ['MS Wade', 132, 123, 9, 14.666666666666666, 107.31707317073172],
 ['HH Pandya', 1226, 839, 37, 33.13513513513514, 146.12634088200238],
 ['DA Miller', 1073, 826, 43, 24.953488372093023, 129.90314769975785],
 ['V Kohli', 4587, 3623, 127, 36.118110236220474, 126.60778360474745],
 ['F du P

In [54]:
batting_power_play_df = pd.DataFrame(batting_power_play, columns=['Batter', 'runs', 'balls', 'Dismissals', 'average', 'strike_rate'])
batting_power_play_df

,Batter,runs,balls,Dismissals,average,strike_rate
0,YBK Jaiswal,452,337,19,23.789474,134.124629
1,JC Buttler,2072,1415,51,40.627451,146.431095
2,SV Samson,2250,1689,81,27.777778,133.214920
3,D Padikkal,1089,852,38,28.657895,127.816901
4,SO Hetmyer,547,384,19,28.789474,142.447917
...,...,...,...,...,...,...
600,D Kalyankrishna,3,8,1,3.000000,37.500000
601,MA Khote,7,10,0,inf,70.000000
602,SB Joshi,3,8,1,3.000000,37.500000
603,DS Lehmann,5,5,1,5.000000,100.000000


In [55]:
sorted_batting_powerplay_df = batting_power_play_df.sort_values('runs', ascending=False)
filtered_sorted_batting_powerplay_df = sorted_batting_powerplay_df.query('runs >= 500').dropna()
print(len(filtered_sorted_batting_powerplay_df))
filtered_sorted_batting_powerplay_df

97


,Batter,runs,balls,Dismissals,average,strike_rate
43,S Dhawan,4942,3905,141,35.049645,126.555698
14,V Kohli,4587,3623,127,36.118110,126.607784
50,DA Warner,4565,3261,105,43.476190,139.987734
59,RG Sharma,4098,3109,138,29.695652,131.810872
179,SK Raina,3993,2895,122,32.729508,137.927461
...,...,...,...,...,...,...
29,E Lewis,545,390,20,27.250000,139.743590
302,HM Amla,535,370,12,44.583333,144.594595
211,BA Stokes,529,416,22,24.045455,127.163462
332,JA Morkel,528,391,26,20.307692,135.038363


In [56]:
# Reset index of filtered_sorted_batting_powerplay_df
filtered_sorted_batting_powerplay_df = filtered_sorted_batting_powerplay_df.reset_index(drop=True)
len(filtered_sorted_batting_powerplay_df)

97

In [57]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Normalize the data
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(filtered_sorted_batting_powerplay_df[['runs', 'average', 'strike_rate']])

# Define weights for the composite score
runs_weight = 0.1
avg_weight = 0.5
strike_rate_weight = 0.7

# Create a new column for the composite score
filtered_sorted_batting_powerplay_df['performance'] = (
    runs_weight * normalized_data[:, 0] +
    avg_weight * normalized_data[:, 1] +
    strike_rate_weight * normalized_data[:, 2]
)

# Split the data into features and target variable
X = normalized_data
y = filtered_sorted_batting_powerplay_df["performance"]


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(3,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Make predictions
predictions = model.predict(X_test)

Epoch 1/100
3/3 [==============================] - 2s 6ms/step - loss: 0.6935 - accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6916 - accuracy: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6904 - accuracy: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6890 - accuracy: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6879 - accuracy: 0.0000e+00
Epoch 6/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6867 - accuracy: 0.0000e+00
Epoch 7/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6857 - accuracy: 0.0000e+00
Epoch 8/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6844 - accuracy: 0.0000e+00
Epoch 9/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6830 - accuracy: 0.0000e+00
Epoch 10/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6815 - accuracy: 

In [58]:
predictions

array([[0.53762734],
       [0.5190409 ],
       [0.41379124],
       [0.4904934 ],
       [0.5058999 ],
       [0.7469248 ],
       [0.2885794 ],
       [0.521159  ],
       [0.37157127],
       [0.41176766],
       [0.6571164 ],
       [0.71251625],
       [0.27768436],
       [0.50609475],
       [0.32831797],
       [0.43866044],
       [0.83334297],
       [0.22740538],
       [0.3979144 ],
       [0.2773272 ]], dtype=float32)

In [59]:
len(predictions)

20

In [60]:
output_players_performance = model.predict(X)
print(output_players_performance)

4/4 [==============================] - 0s 3ms/step
[[0.5504581 ]
 [0.5607199 ]
 [0.7787642 ]
 [0.5058999 ]
 [0.6108366 ]
 [0.78351957]
 [0.83334297]
 [0.38420808]
 [0.36880982]
 [0.39790922]
 [0.69003856]
 [0.8135299 ]
 [0.4823677 ]
 [0.5834626 ]
 [0.41379124]
 [0.41554475]
 [0.43314114]
 [0.4628069 ]
 [0.28431243]
 [0.44442162]
 [0.55962074]
 [0.3300861 ]
 [0.57338226]
 [0.7582461 ]
 [0.41176766]
 [0.38646823]
 [0.5370448 ]
 [0.5634988 ]
 [0.44192156]
 [0.31609148]
 [0.521159  ]
 [0.37779218]
 [0.41315553]
 [0.39654353]
 [0.71251625]
 [0.51073307]
 [0.20903121]
 [0.28048864]
 [0.3979144 ]
 [0.3085466 ]
 [0.5222543 ]
 [0.6806506 ]
 [0.55220145]
 [0.39594352]
 [0.39888427]
 [0.5585634 ]
 [0.49582046]
 [0.41866714]
 [0.43866044]
 [0.5138027 ]
 [0.34442765]
 [0.6403127 ]
 [0.20477904]
 [0.2773272 ]
 [0.16878174]
 [0.37157127]
 [0.3028027 ]
 [0.33713153]
 [0.9112744 ]
 [0.3404212 ]
 [0.6525308 ]
 [0.32831797]
 [0.7609172 ]
 [0.36140913]
 [0.14882591]
 [0.5582383 ]
 [0.2885794 ]
 [0.3811038

In [61]:
len(output_players_performance)

97

In [62]:
filtered_sorted_batting_powerplay_df

,Batter,runs,balls,Dismissals,average,strike_rate,performance
0,S Dhawan,4942,3905,141,35.049645,126.555698,0.549509
1,V Kohli,4587,3623,127,36.118110,126.607784,0.556995
2,DA Warner,4565,3261,105,43.476190,139.987734,0.780291
3,RG Sharma,4098,3109,138,29.695652,131.810872,0.502246
4,SK Raina,3993,2895,122,32.729508,137.927461,0.597534
...,...,...,...,...,...,...,...
92,E Lewis,545,390,20,27.250000,139.743590,0.458737
93,HM Amla,535,370,12,44.583333,144.594595,0.746148
94,BA Stokes,529,416,22,24.045455,127.163462,0.300250
95,JA Morkel,528,391,26,20.307692,135.038363,0.318336


# Best Batters In the Powerplay

In [63]:
import pandas as pd

# Create a new dataframe from the array
performance_df = pd.DataFrame(output_players_performance, columns=['Actual Performance'])

# Concatenate the two dataframes along the column axis
merged_df = pd.concat([filtered_sorted_batting_powerplay_df, performance_df], axis=1)
merged_df.sort_values('Actual Performance', ascending=False)

,Batter,runs,balls,Dismissals,average,strike_rate,performance,Actual Performance
58,AD Russell,1063,579,30,35.433333,183.592401,0.979554,0.911274
6,AB de Villiers,3475,2248,85,40.882353,154.581851,0.850219,0.833343
11,KL Rahul,2806,2073,54,51.962963,135.359383,0.818445,0.813530
5,CH Gayle,3979,2684,101,39.396040,148.248882,0.783799,0.783520
2,DA Warner,4565,3261,105,43.476190,139.987734,0.780291,0.778764
...,...,...,...,...,...,...,...,...
86,IK Pathan,574,503,26,22.076923,114.115308,0.156342,0.169682
54,NV Ojha,1102,960,55,20.036364,114.791667,0.145623,0.168782
85,MS Bisla,583,518,26,22.423077,112.548263,0.147344,0.163660
89,Y Venugopal Rao,562,506,26,21.615385,111.067194,0.122192,0.149074
